In [16]:
import nltk
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [17]:
beers=pd.read_csv('beer_list2.csv')

In [18]:
beers.head()

,Unnamed: 0,id,name,abv,ibu,original_gravity,is_organic,category_id,category_name,style_id,style_name,style_description,seriving_temp,food_pairing,description,label
0,0,njehqH,144 IPA,NaN,0,NaN,NaN,3.0,North American Origin Ales,30.0,American-Style India Pale Ale,American-style India pale ales are perceived t...,NaN,NaN,NaN,NaN
1,1,EVVV1r,1488 Whisky Beer,7.0,0,NaN,NaN,11.0,Hybrid/mixed Beer,138.0,Other Strong Ale or Lager,Any style of beer can be made stronger than th...,NaN,NaN,NaN,NaN
2,2,295GZo,19 Original Colonies,5.5,0,1.008,NaN,12.0,"Mead, Cider, & Perry",148.0,Open Category Mead,This mead should exhibit the character of all ...,NaN,NaN,19 Original Colonies Mead is brewed using 5 in...,https://s3.amazonaws.com/brewerydbapi/beer/295...
3,3,Np1soi,666 Premium British Beer,6.6,0,NaN,NaN,1.0,British Origin Ales,3.0,Ordinary Bitter,Ordinary bitter is gold to copper colored with...,NaN,NaN,NaN,NaN
4,4,7xU7NH,Ace of Simcoe,4.5,0,NaN,NaN,3.0,North American Origin Ales,164.0,Session India Pale Ale,Session India Pale Ales are gold to copper. Ch...,NaN,NaN,NaN,NaN


In [19]:
beers.shape

(19494, 16)

In [20]:
beers_test = beers.loc[:10000,['description','style_id']]
beers_test.head()

,description,style_id
0,NaN,30.0
1,NaN,138.0
2,19 Original Colonies Mead is brewed using 5 in...,148.0
3,NaN,3.0
4,NaN,164.0


In [21]:
beers_test.isnull().sum()

description    1790
style_id         47
dtype: int64

In [22]:
beers_fun = beers_test.dropna()

In [8]:
X = beers_fun.description
y = beers_fun.style_id
print y.value_counts().head()


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

72.0    438
36.0    404
32.0    309
25.0    281
48.0    271
Name: style_id, dtype: int64


In [16]:
# use default options for CountVectorizer
vect = CountVectorizer(stop_words='english')

# create document-term matrices :: only single words
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use logistic regression with document feature matrix, NOT the text column
logreg = LogisticRegression()
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)

# calculate accuracy
print metrics.accuracy_score(y_test, y_pred_class)

0.437102300538


In [20]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
logreg = LogisticRegression()
xx = vect.fit_transform(X)

scores = cross_val_score(logreg, xx, y, cv=5, scoring='recall')
scores

C:\Users\antho\Anaconda2\envs\data28\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
C:\Users\antho\Anaconda2\envs\data28\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of sco

array([ 0.39294118,  0.44078947,  0.46196319,  0.42418546,  0.44853875])

In [21]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
preds = nb.predict(X_test_dtm)
from sklearn import metrics
print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)

0.209006363191
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [24]:
mean = cross_val_score(nb, xx, y, cv=10, scoring='accuracy').mean()

C:\Users\antho\Anaconda2\envs\data28\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


In [25]:
mean

0.24323419108201075

In [38]:
from collections import Counter
words = []

def word_count(row):
    return Counter(row)

words += [word_count(r) for r in beers_fun['description']]
       

In [39]:
words

[Counter({' ': 34,
          '&': 1,
          ',': 4,
          '.': 2,
          '1': 1,
          '5': 1,
          '9': 1,
          ':': 1,
          'C': 4,
          'F': 2,
          'G': 1,
          'H': 3,
          'J': 1,
          'L': 1,
          'M': 1,
          'N': 1,
          'O': 1,
          'R': 2,
          'S': 1,
          'T': 2,
          'W': 2,
          'Y': 1,
          'a': 16,
          'b': 1,
          'c': 1,
          'd': 8,
          'e': 31,
          'g': 6,
          'h': 2,
          'i': 15,
          'l': 7,
          'm': 3,
          'n': 11,
          'o': 10,
          'p': 4,
          'r': 13,
          's': 13,
          't': 6,
          'u': 3,
          'v': 5,
          'w': 3,
          'y': 3}),
 Counter({' ': 42,
          '.': 3,
          'B': 3,
          'T': 2,
          'W': 1,
          'a': 17,
          'c': 5,
          'd': 8,
          'e': 25,
          'f': 4,
          'g': 2,
          'h': 13,
          'i':

In [37]:
?Counter

In [25]:
X = beers_fun.description
y = beers_fun.style_id
vect = CountVectorizer()
xx = vect.fit_transform(X)
logreg = LogisticRegression()
filter1 = SelectKBest(k=2) # select the best 2 features
pipe = Pipeline([('anova', filter1), ('logistic', logreg)])


mean = cross_val_score(pipe, xx, y, cv=10, scoring='accuracy').mean()
mean

C:\Users\antho\Anaconda2\envs\data28\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
C:\Users\antho\Anaconda2\envs\data28\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 

0.053754876776035007